In [2]:
import re
from google.cloud import storage

def find_flight_image_folders(path, project_id):
    """
    Finds flight data folders in a GCS path.
    """
    if not path.startswith('gs://'):
        raise ValueError("This function is for GCS paths.")

    print(f"Listing folders in GCS path: {path}")
    date_regex = re.compile(r'^\d{8}')
    matching_folders = []

    try:
        storage_client = storage.Client(project=project_id)
        bucket_name = "data-uas"  # From your screenshot
        # --- KEY CHANGE: Use the correct and full prefix ---
        prefix_to_list = "2025_poc/flights/"
        
        bucket = storage_client.bucket(bucket_name)

        iterator = bucket.list_blobs(prefix=prefix_to_list, delimiter='/')
        first_level_prefixes = list(iterator.prefixes)

        for first_level_prefix in first_level_prefixes:
            # e.g., first_level_prefix is "2025_poc/flights/20250430-Wooster/"
            first_level_folder = first_level_prefix.rstrip('/').split('/')[-1]

            if date_regex.match(first_level_folder.split('-')[0]):
                # Now list the sub-folders (e.g., "20250430_Wooster_Altum_.../")
                sub_iterator = bucket.list_blobs(prefix=first_level_prefix, delimiter='/')
                for second_level_prefix in list(sub_iterator.prefixes):
                    second_level_folder = second_level_prefix.rstrip('/').split('/')[-1]
                    final_path = f"gs://{bucket_name}/{second_level_prefix}01_Images/{second_level_folder}/"
                    matching_folders.append(final_path)

        print(f"Found {len(matching_folders)} matching folders in GCS.")
        return matching_folders

    except Exception as e:
        print(f"Error accessing GCS path '{path}': {e}")
        return []

In [45]:
import re
from google.cloud import storage
from google.oauth2 import service_account

def test_explicit_credentials():
    """
    A self-contained function to test GCS listing with an explicit service account key.
    """
    project_id = 'uas-orchestration-engine'
    bucket_name = 'data-uas'
    prefix_to_list = '2025_poc/flights/'
    
    # --- IMPORTANT: Update this path to your key file ---
    key_file_path = "/Users/lukewaltz/.gcp/uas-orchestration-engine-1ddb7eebd83f.json"
    
    print("--- Running Test with Explicit Key File ---")
    print(f"Key file: {key_file_path}")
    print(f"Project: {project_id}")
    print(f"Bucket: {bucket_name}")
    print(f"Prefix: {prefix_to_list}")
    
    try:
        # 1. Explicitly load credentials from the file
        credentials = service_account.Credentials.from_service_account_file(key_file_path)
        
        # 2. Create the client, passing in the explicit credentials
        storage_client = storage.Client(project=project_id, credentials=credentials)
        bucket = storage_client.get_bucket(bucket_name)
        
        # 3. List the "subdirectories" at the specified prefix
        iterator = bucket.list_blobs(prefix=prefix_to_list, delimiter='/')
        subdirectories = list(iterator.prefixes)
        
        print("\n--- Results ---")
        if not subdirectories:
            print("Result: No subdirectories found.")
        else:
            print("Result: Success! Found the following subdirectories:")
            for subdir in subdirectories:
                print(f"- {subdir}")

    except Exception as e:
        print(f"\n--- An Error Occurred ---")
        print(e)

# Run the test function
test_explicit_credentials()

--- Running Test with Explicit Key File ---
Key file: /Users/lukewaltz/.gcp/uas-orchestration-engine-1ddb7eebd83f.json
Project: uas-orchestration-engine
Bucket: data-uas
Prefix: 2025_poc/flights/

--- An Error Occurred ---
403 GET https://storage.googleapis.com/storage/v1/b/data-uas?projection=noAcl&prettyPrint=false: uas-storage-objectadmin@uas-orchestration-engine.iam.gserviceaccount.com does not have storage.buckets.get access to the Google Cloud Storage bucket. Permission 'storage.buckets.get' denied on resource (or it may not exist).


In [ ]:
def test_gcs_listing_final():
    """
    A self-contained function to test GCS listing,
    explicitly setting the user_project when creating the bucket handle.
    """
    project_id = 'uas-orchestration-engine'
    bucket_name = 'data-uas'
    prefix_to_list = '2025_poc/flights/'
    
    print(f"--- Running Final Test ---")
    
    try:
        storage_client = storage.Client(project=project_id)
        
        # --- KEY CHANGE: Specify user_project when creating the bucket handle ---
        bucket = storage_client.bucket(bucket_name, user_project=project_id)
        
        # Now, list the blobs. This operation will inherit the user_project setting.
        iterator = list(bucket.list_blobs(prefix=prefix_to_list, delimiter='/'))
        
        print(f'iterator: {iterator}')
        #subdirectories = list(iterator.prefixes)
        
        print("\n--- Results ---")
        if not subdirectories:
            print("Result: No subdirectories found.")
        else:
            print("Result: Success! Found the following subdirectories:")
            for subdir in subdirectories:
                print(f"- {subdir}")

    except Exception as e:
        print(f"\n--- An Error Occurred ---")
        print(e)

# Run the test function
test_gcs_listing_final()

--- Running Final Test ---
iterator: [<Blob: data-uas, 2025_poc/flights/20250430-Wooster/20250430_Wooster_Replant_Altum233 Flight 01/01_Images/20250430_Wooster_Replant_Altum233 Flight 01/DATA/0111120a.25o, 1756398471722243>, <Blob: data-uas, 2025_poc/flights/20250430-Wooster/20250430_Wooster_Replant_Altum233 Flight 01/01_Images/20250430_Wooster_Replant_Altum233 Flight 01/DATA/20250430_Wooster_Replant_Altum233 Flight 01 Base.sbf, 1756398471006121>, <Blob: data-uas, 2025_poc/flights/20250430-Wooster/20250430_Wooster_Replant_Altum233 Flight 01/01_Images/20250430_Wooster_Replant_Altum233 Flight 01/DATA/20250430_Wooster_Replant_Altum233 Flight 01 WingtraOne ForwardProcessed.sbf, 1756398471016227>, <Blob: data-uas, 2025_poc/flights/20250430-Wooster/20250430_Wooster_Replant_Altum233 Flight 01/01_Images/20250430_Wooster_Replant_Altum233 Flight 01/DATA/20250430_Wooster_Replant_Altum233 Flight 01 WingtraOne.sbf, 1756398471504639>, <Blob: data-uas, 2025_poc/flights/20250430-Wooster/20250430_Woost

In [43]:
from google.cloud import storage

project_id = 'uas-orchestration-engine'
bucket_name = 'data-uas'

storage_client = storage.Client(project=project_id)
bucket = storage_client.bucket(bucket_name, user_project=project_id)

# Test basic bucket access
try:
    bucket.get_iam_policy()  # Requires storage.buckets.getIamPolicy permission
    print(f"Successfully accessed bucket: {bucket_name}")
    # List a few objects to confirm access
    blobs = list(bucket.list_blobs(max_results=5))
    print(f"Sample objects in bucket: {[blob.name for blob in blobs]}")
except Exception as e:
    print(f"Error accessing bucket {bucket_name}: {e}")

Successfully accessed bucket: data-uas
Sample objects in bucket: ['2025_poc/config/uas_config.yaml', '2025_poc/flights/20250430-Wooster/20250430_Wooster_Replant_Altum233 Flight 01/01_Images/20250430_Wooster_Replant_Altum233 Flight 01/DATA/0111120a.25o', '2025_poc/flights/20250430-Wooster/20250430_Wooster_Replant_Altum233 Flight 01/01_Images/20250430_Wooster_Replant_Altum233 Flight 01/DATA/20250430_Wooster_Replant_Altum233 Flight 01 Base.sbf', '2025_poc/flights/20250430-Wooster/20250430_Wooster_Replant_Altum233 Flight 01/01_Images/20250430_Wooster_Replant_Altum233 Flight 01/DATA/20250430_Wooster_Replant_Altum233 Flight 01 WingtraOne ForwardProcessed.sbf', '2025_poc/flights/20250430-Wooster/20250430_Wooster_Replant_Altum233 Flight 01/01_Images/20250430_Wooster_Replant_Altum233 Flight 01/DATA/20250430_Wooster_Replant_Altum233 Flight 01 WingtraOne.sbf']
